In [3]:
from nbfinder import NotebookFinder
import sys
sys.meta_path.append(NotebookFinder())
sys.path.append("..")
import numpy as np
from box_processing.get_iou import get_iou

def get_boxes_of_class(boxes, cls):
    cls_ind = 5
    boxes = [box for box in boxes if box[cls_ind] == cls]
    return boxes

def reverse_confidences(boxes,confidence_index=4):
    for ind, box in enumerate(boxes):
        boxes[ind][confidence_index] = 1 -  box[confidence_index]
    return boxes
        
def sort_boxes_by_confidence(boxes, confidence_index=4):
    boxes = [box for box in boxes if box[confidence_index] > 0.0]
    boxes.sort(lambda a,b: -1 if a[confidence_index] > b[confidence_index] else 1)
    return boxes

def calc_true_and_false_positives_for_an_example(box_pred, box_gt, cls, iou_thresh, reverse_conf=False):    
    #box_pred and box_gt are lists 
    #filter for class

    box_pred = get_boxes_of_class(box_pred, cls)
    box_gt = get_boxes_of_class(box_gt, cls)

    if reverse_conf:
        box_pred = reverse_confidences(box_pred)
        
    #sort by confidence
    confidence_index = 4
    box_pred = sort_boxes_by_confidence(box_pred, confidence_index=confidence_index)

    #boolean array for whether gt box has been matched yet
    num_gt_boxes = len(box_gt)
    gt_box_picked_yet = np.zeros(num_gt_boxes)

    y_true = []
    y_score = []

    #for each box with highest confidence first, 
    #find the first gt_box
    for box_ind, box_p in enumerate(box_pred):
        conf = box_p[confidence_index]
        found_match = False
        for gt_box_ind, box_g in enumerate(box_gt):
            iou = get_iou(box_p, box_g)
            
#             if iou > 0:
#                 print box_p, box_g, iou, iou_thresh,gt_box_picked_yet[gt_box_ind] 
            
            
            # if gt box not matched and iou is big enough -> true positive
            # and break out of inner loop
            
            if iou >= iou_thresh and not gt_box_picked_yet[gt_box_ind]:
                #true postive! -> add confidence to y_score list
                y_score.append(conf)
                y_true.append(1.)
                gt_box_picked_yet[gt_box_ind] = 1.
                found_match = True
                
                #stop looking. go to next box
                break
                
            # if not keep looking
            else:
                continue
        
        # if inner loop finished and still haven't found a gt box to match to
        # then we are a false positive
        if not found_match:
            y_score.append(conf)
            y_true.append(0.)

    return y_true, y_score   

importing Jupyter notebook from ../box_processing/get_iou.ipynb
importing Jupyter notebook from ../box_processing/convert_between_xy_minmax_xy_center_boxes.ipynb


In [3]:

#! jupyter nbconvert --to script compute_score.ipynb

[NbConvertApp] Converting notebook compute_score.ipynb to script
[NbConvertApp] Writing 3739 bytes to compute_score.py
